In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import fetch_20newsgroups


In [2]:
newsgroups = fetch_20newsgroups(subset = 'train', remove = ('headers', 'footers', 'quotes'))
documents = newsgroups.data

In [5]:
newsgroup_test = fetch_20newsgroups(subset = 'test', remove = ('headers', 'footers', 'quotes'))

In [6]:
len(newsgroup_test)
test_docs = newsgroup_test.data
len(test_docs)

7532

In [3]:
newsgroups.target

array([7, 4, 4, ..., 3, 1, 8])

In [12]:
newsgroups.filenames.shape
newsgroups.target_names, newsgroups.target
target_names = newsgroups.target_names
target_name_mapping = {}
for i, name in enumerate(target_names):
  target_name_mapping[name] = i


In [13]:
target_name_mapping

{'alt.atheism': 0,
 'comp.graphics': 1,
 'comp.os.ms-windows.misc': 2,
 'comp.sys.ibm.pc.hardware': 3,
 'comp.sys.mac.hardware': 4,
 'comp.windows.x': 5,
 'misc.forsale': 6,
 'rec.autos': 7,
 'rec.motorcycles': 8,
 'rec.sport.baseball': 9,
 'rec.sport.hockey': 10,
 'sci.crypt': 11,
 'sci.electronics': 12,
 'sci.med': 13,
 'sci.space': 14,
 'soc.religion.christian': 15,
 'talk.politics.guns': 16,
 'talk.politics.mideast': 17,
 'talk.politics.misc': 18,
 'talk.religion.misc': 19}

In [ ]:
type(documents), len(documents)

(list, 18846)

In [ ]:
documents[0]

"\n\nI am sure some bashers of Pens fans are pretty confused about the lack\nof any kind of posts about the recent Pens massacre of the Devils. Actually,\nI am  bit puzzled too and a bit relieved. However, I am going to put an end\nto non-PIttsburghers' relief with a bit of praise for the Pens. Man, they\nare killing those Devils worse than I thought. Jagr just showed you why\nhe is much better than his regular season stats. He is also a lot\nfo fun to watch in the playoffs. Bowman should let JAgr have a lot of\nfun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway. I was very disappointed not to see the Islanders lose the final\nregular season game.          PENS RULE!!!\n\n"

In [ ]:
# tokeninzation with BERT
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
tokens = [tokenizer.tokenize(doc) for doc in documents]

Token indices sequence length is longer than the specified maximum sequence length for this model (1572 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
# tokens[0:3]

In [ ]:
max_length = 512
input_ids = [tokenizer.encode(doc, add_special_tokens= True, max_length = max_length, pad_to_max_length = True) for doc in documents]

type(input_ids)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2606: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


list

In [ ]:
! pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 78.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 40.1 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModel, AutoTokenizer
import torch

# Initialize BERT model and tokenizer
model = AutoModel.from_pretrained("bert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Define a list of documents (list of strings)
# documents = ["Your list of documents goes here", "Another document", ...]

# Tokenize and create embeddings for each document
embeddings = []
device = "cuda" if torch.cuda.is_available() else "cpu"
for doc in documents[:5]:
    # Tokenize the document and convert to input IDs
    input_ids = tokenizer.encode(doc, add_special_tokens=True)
    input_ids = torch.tensor(input_ids).unsqueeze(0)  # Add batch dimension

    # Get BERT embeddings
    with torch.no_grad():
        output = model(input_ids)
        embeddings.append(output.last_hidden_state.mean(dim=1).numpy())

# 'embeddings' now contains BERT embeddings for each document


ModuleNotFoundError: ignored

In [ ]:

from transformers import AutoModel, AutoTokenizer
import torch


max_length = 512
embeddings = []

# Initialize BERT model and tokenizer
model = AutoModel.from_pretrained("bert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
for doc in documents[:5]:
  tokens = tokenizer.tokenize(doc)[:max_length-2]

  input_ids = tokenizer.convert_tokens_to_ids(tokens)
  input_ids = [tokenizer.cls_token_id] + input_ids + [tokenizer.sep_token_id]
  input_ids = torch.tensor(input_ids).unsqueeze(0)

  with torch.no_grad():
    output = model(input_ids)
    embeddings.append(output.last_hidden_state.mean(dim =1).numpy())

In [ ]:
embeddings

[array([[-1.21342070e-01, -1.69136882e-01,  8.87631997e-02,
          1.96189731e-01, -6.43044412e-02, -3.02438587e-01,
         -2.18970440e-02,  6.67981446e-01, -1.44535556e-01,
         -4.37842846e-01,  1.92853332e-01, -4.90436345e-01,
         -1.69584244e-01,  2.51314968e-01, -3.35879549e-02,
          5.12562394e-01, -2.03782693e-02,  9.22629908e-02,
         -3.29109803e-02,  4.89312977e-01,  4.62825269e-01,
         -1.69899404e-01,  1.72779575e-01,  6.88461542e-01,
          2.67496794e-01, -2.75461818e-03, -7.04748556e-04,
          3.63601707e-02, -3.04527164e-01, -8.01734552e-02,
          4.63973016e-01, -2.62731612e-02, -2.84288049e-01,
         -1.61357731e-01, -2.96412166e-02, -3.29836249e-01,
         -1.95686221e-01, -7.38804713e-02, -6.46396577e-02,
          1.77768350e-01, -5.92639744e-01, -2.98366308e-01,
          8.74859467e-03,  5.79199232e-02, -1.32552847e-01,
         -2.30463654e-01,  6.10264897e-01,  1.42111525e-01,
          2.08808467e-01, -1.71443999e-0